https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization

In [2]:
import functools
import os
CWD = os.getcwd()
from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import shutil
import tensorflow as tf
import keras
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))
print(CWD)

TF Version:  2.15.0
TF Hub version:  0.15.0
Eager mode enabled:  True
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/mnt/e/APResearch/datasets


In [3]:
# Load TF Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

2024-02-07 16:48:10.292843: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:48:10.292916: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:48:10.292946: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:48:10.702171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:48:10.702217: I external/local_xla/xla/stream_executor

In [4]:
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(PATH, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(PATH),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [5]:
def stylize(content_image, style_image):
    outputs = hub_module(content_image, style_image)
    stylized_image = outputs[0]
    # Stylize content image with given style image.
    # This is pretty fast within a few milliseconds on a GPU.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    # print(outputs)
    stylized_image = outputs[0]
    # Visualize input images and the generated stylized image.
    # show_n([content_image, style_image, stylized_image], titles=['Original content image', 'Style image', 'Stylized image'])
    stylized_image = tf.squeeze(stylized_image)
    return stylized_image

Randomly Select 185 ISIC_2017 benign images for content - benign and 18 ISIC_2017 malignant images for content - malignant.

In [7]:
import random
def choose(min, max, count):
    indexes = set()
    while len(indexes) < count:
        i = random.randint(min,max)
        indexes.add(i)    
    return indexes

In [10]:
benign = os.listdir(f'{CWD}/ISIC_2017/benign')
print(len(benign))
indexes_benign = choose(0, 374, 185)
for index in indexes_benign:
    shutil.copy(f'{CWD}/ISIC_2017/benign/{benign[index]}', f'{CWD}/ArGI_2024_content_images/benign/{benign[index]}')

malignant = os.listdir(f'{CWD}/ISIC_2017/malignant')
print(len(malignant))
indexes_malignant = choose(0, 520, 18)
for index in indexes_malignant:
    shutil.copy(f'{CWD}/ISIC_2017/malignant/{malignant[index]}', f'{CWD}/ArGI_2024_content_images/malignant/{malignant[index]}')

375
520


Stylize the Content Images

In [11]:
style_images = os.listdir(f'{CWD}/ArGI_2024_style_images')
# Generate benign images
content_benign = os.listdir(f'{CWD}/ArGI_2024_content_images/benign')
for i in range(185):
    idx = random.randint(0,43)
    content_img = content_benign[i]
    style_img = style_images[idx]

    style_image = load_image(f"{CWD}/ArGI_2024_style_images/{style_img}", (224,224))
    content_image = load_image(f'{CWD}/ArGI_2024_content_images/benign/{content_img}', (256, 256))

    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
    keras.utils.save_img(f'{CWD}/ArGI_2024/benign/{content_img}_stylized_with_{style_img}.jpg', 
                         stylize(content_image, style_image))

content_malignant = os.listdir(f'{CWD}/ArGI_2024_content_images/malignant')
for i in range(18):
    idx = random.randint(0,43)
    content_img = content_malignant[i]
    style_img = style_images[idx]

    style_image = load_image(f"{CWD}/ArGI_2024_style_images/{style_img}", (224,224))
    content_image = load_image(f'{CWD}/ArGI_2024_content_images/malignant/{content_img}', (256, 256))

    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
    keras.utils.save_img(f'{CWD}/ArGI_2024/malignant/{content_img}_stylized_with_{style_img}.jpg', 
                         stylize(content_image, style_image))

2024-02-07 17:26:49.516669: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2024-02-07 17:26:49.971549: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-07 17:26:51.200649: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2024-02-07 17:26:54.202973: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-07 17:26:54.259198: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more me